In [1]:
!pip install gradio dashscope

In [5]:
import os
import gradio as gr
import dashscope
from dashscope import MultiModalConversation
from PIL import Image

# ================= 1. 配置与风格定义 =================
if "DASHSCOPE_API_KEY" not in os.environ:
    os.environ["DASHSCOPE_API_KEY"] = "sk-xxxxxxxxxxxxxxxx" # 替换为你的真实 Key

STYLE_PROMPTS = {
    "默认 (Default)": "",
    "水墨国风 (Chinese Ink)": ", 中国风, 水墨画风格, 典雅庄重, 留白",
    "赛博朋克 (Cyberpunk)": ", 赛博朋克风格, 霓虹灯光, 机械感, 未来主义",
    "日系动漫 (Anime)": ", 二次元风格, 动漫插画, 赛璐璐, 鲜艳色彩",
    "写实摄影 (Photography)": ", 4K高清, 真实摄影, 电影质感, 极其逼真",
    "3D 渲染 (3D Render)": ", 3D渲染, C4D, 虚幻引擎, 极简主义",
    "油画 (Oil Painting)": ", 油画风格, 印象派, 笔触细腻, 艺术感"
}

SIZE_OPTIONS = [
    ("1:1 方形 (1328*1328)", "1328*1328"),
    ("16:9 横屏 (1664*928)", "1664*928"),
    ("4:3 横屏 (1472*1140)", "1472*1140"),
    ("3:4 竖屏 (1140*1472)", "1140*1472"),
    ("9:16 竖屏 (928*1664)", "928*1664")
]

# ================= 2. 适配 Sketch 主题的自定义 CSS =================
# Sketch 主题自带手绘线条感，这里的 CSS 主要用于增强按钮和标题的视觉
custom_css = """
#title-header { 
    text-align: center; 
    padding: 20px; 
    margin-bottom: 20px; 
    border-bottom: 2px dashed #555; 
}
.generate-btn { 
    border: 2px solid #333 !important;
    box-shadow: 3px 3px 0px #333 !important;
    background-color: #fff !important;
    color: #333 !important;
    font-weight: bold !important;
}
.generate-btn:hover { 
    transform: translate(-1px, -1px);
    box-shadow: 5px 5px 0px #333 !important;
}
.generate-btn:active {
    transform: translate(2px, 2px);
    box-shadow: 0px 0px 0px #333 !important;
}
"""

# ================= 3. 核心逻辑 =================
def call_qwen_api(prompt, size):
    messages = [{"role": "user", "content": [{"text": prompt}]}]
    try:
        response = MultiModalConversation.call(
            model="qwen-image-plus",
            messages=messages,
            result_format='message',
            size=size,
            prompt_extend=True
        )
        if response.status_code == 200:
            content_list = response.output.choices[0].message.content
            img_url = next((item['image'] for item in content_list if 'image' in item), None)
            return True, img_url
        return False, f"API 错误: {response.message}"
    except Exception as e:
        return False, str(e)

def generate_image_handler(prompt, style_name, size_val, num_images, progress=gr.Progress()):
    if not prompt.strip(): return None, "❌ 提示词不能为空"

    final_prompt = prompt + STYLE_PROMPTS.get(style_name, "")
    results, logs = [], []

    for i in range(int(num_images)):
        progress(float(i+1)/num_images, desc=f"正在创作中...")
        success, res = call_qwen_api(final_prompt, size_val)
        if success:
            results.append(res)
            logs.append(f"✅ 第 {i+1} 张完成")
        else:
            logs.append(f"❌ 第 {i+1} 张失败: {res}")

    return results, "\n".join(logs)

# ================= 4. 构建界面 =================
with gr.Blocks() as demo:
    with gr.Column(elem_id="title-header"):
        gr.Markdown("# ✏️ Sketch AI 画室")
        gr.Markdown("基于 Qwen-Image-Plus · 让灵感跃然纸上")

    with gr.Row():
        # 左侧：配置区
        with gr.Column(scale=4):
            with gr.Group():
                input_prompt = gr.Textbox(
                    label="想要画什么？",
                    placeholder="输入你的描述...",
                    lines=5
                )
                input_style = gr.Dropdown(
                    choices=list(STYLE_PROMPTS.keys()),
                    value="默认 (Default)",
                    label="画风选择"
                )
                input_size = gr.Dropdown(
                    choices=SIZE_OPTIONS,
                    value="1328*1328",
                    label="画布比例"
                )

            with gr.Accordion("更多选项", open=False):
                input_batch = gr.Slider(minimum=1, maximum=4, step=1, value=1, label="批量张数")

            btn_run = gr.Button("🎨 绘制图像", variant="primary", elem_classes="generate-btn")
            output_log = gr.Textbox(label="创作日志", lines=2, interactive=False)

        # 右侧：展示区
        with gr.Column(scale=6):
            output_gallery = gr.Gallery(
                label="作品集",
                columns=2,
                height=600,
                preview=True
            )

            with gr.Row():
                gr.ClearButton(components=[input_prompt, output_gallery], value="🧹 清空画板")

    # 事件绑定
    btn_run.click(
        fn=generate_image_handler,
        inputs=[input_prompt, input_style, input_size, input_batch],
        outputs=[output_gallery, output_log]
    )

# ================= 5. 启动应用 =================
if __name__ == "__main__":
    # 直接引用 Hugging Face 上的主题名
    demo.launch(
        theme="gstaff/sketch",
        css=custom_css
    )

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
